In [1]:
import torch
import torch.nn as nn
from torch.nn import Linear, Module

from torchvision.models import resnet18
from models.stylegan2.model import Generator

/home/aiteam/miniconda3/envs/apa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
enc_t = resnet18(pretrained=False)

In [22]:
class Encoder(nn.Module):
    def __init__(self, emb_space):
        super(Encoder, self).__init__()
        resnet = resnet18(pretrained=False)

        if emb_space == 'tensor':
            self.features = nn.Sequential(*list(resnet.children())[:-2])
        elif emb_space == 'modulation':
            self.features = nn.Sequential(*list(resnet.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x

In [19]:
enc_t = Encoder('tensor') # [1, 3, 256, 256] -> [1, 512, 8, 8] # 논문에선 512 x 4 x 4

In [23]:
enc_w = Encoder('modulation') #[1, 3, 256, 256] -> [1, 512, 1, 1]

In [ ]:
import math
class make_empty_object(object):
  pass


config.input_size = 256
config.stylegan_weight = '/home/aiteam/tykim/generative/gan/3D-FM-GAN/pretraiend/550000.pt'

class FMGAN(nn.Module):
  def __init__(self, config):
    # [1, 3, 256, 256] -> [1, 512, 8, 8] # 논문에선 512 x 4 x 4
    self.enc_t = Encoder('tensor')
    #[1, 3, 256, 256] -> [1, 512, 1, 1]
    self.enc_w = Encoder('modulation')
    opts = make_empty_object()
    opts.input_nc = 3
    output_size = 1024
    opts.n_styles = int(math.log(output_size, 2)) * 2 - 2
    # [1,3,256,256] -> [1, 18, 512]
    self.enc_wplus = GradualStyleEncoder(50, 'ir_se', opts)
    
    self.stylegan = Generator(config.input_size, 512, 8)
    self.stylegan.load_state_dict(ckpt['g_ema'], strict=False)

    
  def forward(self, rendered_img, orig_img):
    t = self.enc_t(rendered_img)
    w = self.enc_w(rendered_img)
    w = w.reshape(1, 1, 512)
    wplus = self.enc_wplus(orig_img)

    w_final = w * wplus 

    # t : 
    # w_final : 
    images, result_latent = self.stylegan([w_final], tensor_input=t,
                                          input_is_latent=True,
                                          randomize_noise=False,
                                          return_latents=True,
                                          input_is_tensor=True)
    # images : [B, 3, 256, 256]
    # result_latent : [B, 18, 512]
    return images

In [2]:
from models.stylegan2.model import Generator

g = Generator(256, 512, 8).to('cuda:1')

In [4]:
test_tensor_input = torch.randn(3, 512, 4, 4).to('cuda:1')
latents = torch.randn(3, 18, 512).to('cuda:1')

In [1]:
import os
import sys

In [2]:
sys.path.append('/home/aiteam/tykim/generative/gan/3D-FM-GAN/models/Deep3DFaceRecon_pytorch')

In [3]:
from options.test_options import TestOptions

/home/aiteam/miniconda3/envs/deep3d_pytorch2/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aiteam/miniconda3/envs/deep3d_pytorch2/lib/python3.6/site-packages/kornia/augmentation/augmentation.py:1875: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


In [ ]:
# from models.Deep3DFaceRecon_pytorch.models import create_model
# from models.Deep3DFaceRecon_pytorch.options.test_options import TestOptions
from models import create_model
from options.test_options import TestOptions


opt = TestOptions(cmd_line=[]).parse()
print(opt)
# model = create_model(opt)

In [ ]:
def train(model, recog_model, recon_loader, dt_loader):
    for i, (p_in, r_in) in enumerate(recon_loader):
        out = model(p_in, r_in)
    